## Materials

- How can we add your own material refractive index info for a particular PDK?
- how can we fail if we try to add a component material that is not in the database?
- How can we make sure different simulation plugins use the same material info? (tidy3d, meep, lumerical ...)

You can define a material by name, real refractive index, complex refractive index (for loss) or by a function of wavelength.

In [ ]:
import numpy as np
import gdsfactory.simulation.gtidy3d as gt

In [ ]:
strip = gt.modes.Waveguide(
    wavelength=1.55,
    wg_width=0.5,
    wg_thickness=0.22,
    slab_thickness=0.0,
    ncore='si',
    nclad='sio2',
)
strip.plot_index()

## Option 1: define material with a value

In [ ]:
from gdsfactory.pdk import _ACTIVE_PDK
_ACTIVE_PDK.materials_index.update(sin=2)

In [ ]:
strip = gt.modes.Waveguide(
    wavelength=1.55,
    wg_width=0.5,
    wg_thickness=0.22,
    slab_thickness=0.0,
    ncore='sin',
    nclad='sio2',
)
strip.plot_index()

## Option 2: define material with a function

In [ ]:
def sin(wav:float)->float:
    w = [1.3, 1.5]
    n = [1.9, 2.1]
    p = np.polyfit(w, n, 1)
    return np.polyval(p, wav)
    
_ACTIVE_PDK.materials_index.update(sin=sin)

In [ ]:
strip = gt.modes.Waveguide(
    wavelength=1.5,
    wg_width=0.5,
    wg_thickness=0.22,
    slab_thickness=0.0,
    ncore='sin',
    nclad='sio2',
)
strip.plot_index()

In [ ]:
from gdsfactory.components.taper import taper_sc_nc
import gdsfactory.simulation.gtidy3d as gt

c = taper_sc_nc(length=10)
c

In [ ]:
s = gt.get_simulation(c, plot_modes=True)
fig = gt.plot_simulation_xz(s)

In [ ]:
import gdsfactory.simulation.gmeep as gm
from gdsfactory.components.taper import taper_sc_nc

c = taper_sc_nc(length=10)
gm.write_sparameters_meep(c, ymargin=3, run=False, wavelength_start=1.31, wavelength_stop=1.36)